In [2]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import os
import subprocess
import re
import pandas as pd
import smtplib
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText


wlfile = "WatchList.csv"
wlfile2 = "WatchList2.csv"
wlfile3 = "EmailWatchList.csv"

Location = wlfile

def format_search_str(word):
    return word.replace(" ", "+")


def output_to_file(lst, filename):
    if os.path.exists(filename) == False:
        with open(filename, "w", newline="") as f:
            csvwriter = csv.writer(f)
            header = ["ASIN", "Title", "Author", "Price", "Link", "WatchList Indicator"]
            csvwriter.writerow(header)
   
    with open(filename, "a", newline="") as f:
        csvwriter = csv.writer(f)
        print(lst)
        csvwriter.writerow(lst)
    
    
def get_url(base_search_url):
    header = {
        "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36"
    }
    resp = requests.get(base_search_url, headers=header).text
    soup = BeautifulSoup(resp,"html.parser")
    return soup



def get_new_price(soup):
    if soup.find("span", {"class" : "a-size-base a-color-price a-color-price"}):
        new_price = soup.find("span", {"class" : "a-size-base a-color-price a-color-price"}).text.strip().replace("$","")
    else:
        if soup.find("span", {"class" : "a-size-base mediaTab_subtitle"}):
            new_price = soup.find("span", {"class" : "a-size-base mediaTab_subtitle"}).text.strip().replace("$","")
        else:
            new_price = "NA"

    return new_price




def main():
    df = pd.read_csv(Location) 

    newdata=[]
    for i, row in df.iterrows():
        soup = get_url(row["Link"])
        new_price = get_new_price(soup)
        row["Price"]  = new_price
        newdata.append(list(row))
        
     
    newdf = pd.DataFrame(data = newdata, columns=["Num","ASIN","Title","Author","Price","Link","WatchList"])
    newdf.to_csv(wlfile2)
    
    mask = newdf["Price"].astype(float) < 7
    newdf = newdf[mask]
    newdf.to_csv(wlfile3)
    msg=newdf

    #prepare to send email
    msg = MIMEMultipart()
    msg["From"] = "dshostmail@gmail.com"
    msg["To"] = "stephie_lsw@yahoo.com"
    msg["Subject"] = "Watchlist"

    ctype, encoding = mimetypes.guess_type(wlfile3)
    if ctype is None or encoding is not None:
        ctype = "application/octet-stream"

    maintype, subtype = ctype.split("/", 1)

    fp = open(wlfile3, "rb")
    attachment = MIMEBase(maintype, subtype)
    attachment.set_payload(fp.read())
    fp.close()
    encoders.encode_base64(attachment)
    
    attachment.add_header("Content-Disposition", "attachment", filename=wlfile3)
    msg.attach(attachment)
    
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()

    server.login("dshostmail@gmail.com", "pwd")
    server.sendmail("dshostmail@gmail.com","user@yahoo.com",msg.as_string())
    server.quit()  


main()
    

    